<a href="https://colab.research.google.com/github/Jorgecardetegit/DiseaseClassifier/blob/main/Model_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Model Deployment

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, InputLayer, BatchNormalization, Input, Dropout, RandomFlip, Resizing, Rescaling
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives, Precision, Recall, AUC
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.regularizers  import L2, L1
from tensorflow.train import Example, Features, Feature, BytesList, FloatList, Int64List

from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import tensorflow_datasets as tfds

from sklearn.metrics import confusion_matrix, roc_curve

from collections import defaultdict
from PIL import Image

import os
import io
import random

ImportError: ignored

### Parameters definition

In [2]:
dataset, info_dataset = tfds.load("malaria", split = ["train"], shuffle_files = True, with_info = True, as_supervised = True)
dataset = dataset[0]

In [3]:
CLASS_NAMES = ["infected", "uninfected"]

CONFIGURATION = {
    "BATCH_SIZE" : 32, "IM_HEIGHT" : 256, "IM_WIDTH" : 256, "NORMALIZATION" : 255, "LEARNING_RATE": 0.001, "N_EPOCHS": 100,
    "DROPOUT_RATE": 0.3, "REGULARIZATION_RATE": 0.0, "N_FILTERS": 6, "KERNEL_SIZE": 3, "N_STRIDES": 1, "POOL_SIZE": 2,
    "N_DENSE_1": 100, "N_DENSE_2": 10, "NUM_CLASSES":2
}

### Rescaling and Normalizing

In [4]:
resize_rescale_layers = Sequential([
    Resizing(CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], interpolation="bilinear"),           # Interpolation method used: Bilinear
    Rescaling(1./CONFIGURATION["NORMALIZATION"])])                                                       # Normalization factor

@tf.function
def map_fn(images, labels):
    processed_images = resize_rescale_layers(images)
    return processed_images, labels

In [5]:
dataset = dataset.map(map_fn, num_parallel_calls = tf.data.AUTOTUNE)

### Data splitting

In [6]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO, seed = None):
  DATASET_SIZE = len(dataset)

  if seed is not None:
    dataset = dataset.shuffle(DATASET_SIZE, seed=seed)
  else:
    dataset = dataset.shuffle(DATASET_SIZE)

  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

In [7]:
train_dataset, val_dataset, test_dataset = splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

### Efficient model storage

In [ ]:
# @title TFRecord files creator

def create_example(image, label):

  bytes_feature = Feature(
      bytes_list=BytesList(value=[image]))

  int_feature = Feature(
      int64_list=Int64List(value=[label]))

  example = Example(
      features=Features(feature={
          'images': bytes_feature,
          'labels': int_feature,
      }))

  return example.SerializeToString()

def encode_image(image, label):
  image = tf.image.convert_image_dtype(image, dtype=tf.uint8)
  image = tf.io.encode_jpeg(image)
  return image,label

encoded_dataset = (
  train_dataset
  .map(encode_image)
)

NUM_SHARDS = 10
PATH = '/content/drive/MyDrive/Computer vision projects/Malaria disease/Malaria dataset/shard_{:02d}.tfrecord'

for shard_number in range(NUM_SHARDS):

  sharded_dataset = (
      encoded_dataset
      .shard(NUM_SHARDS, shard_number)
      .as_numpy_iterator()
  )

  with tf.io.TFRecordWriter(PATH.format(shard_number)) as file_writer:
    for encoded_image, encoded_label in sharded_dataset:

      example = create_example(encoded_image, encoded_label)
      file_writer.write(example)

In [ ]:
# @title TFRecord Dataset creation
recons_dataset = tf.data.TFRecordDataset(
    filenames =[PATH.format(p) for p in range(NUM_SHARDS-2)] )

def parse_tfrecords(example):

    feature_description = {
          "images": tf.io.FixedLenFeature([], tf.string),
          "labels": tf.io.FixedLenFeature([], tf.int64),
      }

    example = tf.io.parse_single_example(example, feature_description)
    example["images"] = tf.image.convert_image_dtype(
        tf.io.decode_jpeg(
        example["images"], channels = 3), dtype = tf.float32)

    return example["images"], example["labels"]

parsed_dataset = (
    recons_dataset
    .map(parse_tfrecords)
    .batch(CONFIGURATION["BATCH_SIZE"])
    .prefetch(tf.data.AUTOTUNE)
)

### Data optimization

In [8]:
train_dataset = (
    train_dataset
    .shuffle(buffer_size = 1024, reshuffle_each_iteration = True)
    .batch(CONFIGURATION["BATCH_SIZE"])
    .prefetch(tf.data.AUTOTUNE)
    )

val_dataset = (
    val_dataset
    .batch(CONFIGURATION["BATCH_SIZE"])
    .prefetch(tf.data.AUTOTUNE)
    )

test_dataset = (
    val_dataset
    .batch(CONFIGURATION["BATCH_SIZE"])
    .prefetch(tf.data.AUTOTUNE)
    )

### Callbacks

### CSV logger

In [9]:
csv_callback = CSVLogger(r"C:\Users\jorge\OneDrive\Imágenes\Documentos\Malaria Detection\Model logs\logs.csv",
                        separator = ",",
                        append = True)

### EarlyStopping

In [10]:
es_callback = EarlyStopping(
    monitor = "val_loss",
    min_delta = 0,                        #An absolute change of less than min_delta, will count as no improvement.
    patience = 5,                         #Number of epochs with no improvement after which training will be stopped.
    verbose = 0,
    mode = "auto",                        #In min mode, training will stop when the quantity monitored has stopped decreasing; in max mode it will stop when the quantity monitored has stopped increasing.
    baseline = None,                      #Baseline value for the monitored quantity. Training will stop if the model doesn't show improvement over the baseline.
    restore_best_weights = False          #Whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used.
)

https://stackoverflow.com/questions/43906048/which-parameters-should-be-used-for-early-stopping

### Learning Rate Scheduler

In [11]:
def scheduler(epoch, lr):
    if epoch <= 1:
        learning_rate = lr
    else:
        learning_rate = lr * tf.math.exp(-0.1)
    return learning_rate

scheduler_callback = LearningRateScheduler(scheduler, verbose = 1)

- https://www.jeremyjordan.me/nn-learning-rate/
- https://datascience.stackexchange.com/questions/410/choosing-a-learning-rate
- https://proceedings.neurips.cc/paper_files/paper/2018/file/a41b3bb3e6b050b6c9067c67f663b915-Paper.pdf


### ReduceLROnPlateau

In [12]:
plateau_callback = ReduceLROnPlateau(monitor = "val_accuracy",
                                     factor = 0.3,
                                     patience = 5,
                                     verbose = 1)

## Model definition

### Custom loss class

In [13]:
class CustomBCE(tf.keras.losses.Loss):
  def __init__(self, FACTOR):
    super(CustomBCE, self).__init__()
    self.FACTOR = FACTOR

  def call(self, y_true, y_pred):
    bce = BinaryCrossentropy()
    return bce(y_true, y_pred)* self.FACTOR

### Custom metrics class

In [14]:
class CustomAccuracy(tf.keras.metrics.Metric):
  def __init__(self, name = 'Custom_Accuracy', FACTOR = 1):
    super(CustomAccuracy, self).__init__()
    self.FACTOR = FACTOR
    self.accuracy = self.add_weight(name = name, initializer = 'zeros')


  def update_state(self, y_true, y_pred, sample_weight = None):
    output = binary_accuracy(tf.cast(y_true, dtype = tf.float32), y_pred)*self.FACTOR
    self.accuracy.assign(tf.math.count_nonzero(output, dtype = tf.float32)/tf.cast(len(output), dtype = tf.float32))

  def result(self):
    return self.accuracy

  def reset_states(self):
    self.accuracy.assign(0.)

FACTOR = 1

### Model metrics definition

In [15]:
metrics = [TruePositives(name='tp'),FalsePositives(name='fp'), TrueNegatives(name='tn'), FalseNegatives(name='fn'),
            BinaryAccuracy(name='accuracy'), Precision(name='precision'), Recall(name='recall'), AUC(name='auc')]

## Model Architecture (LeNet)

### Feature extractor (sequential API)

In [16]:
feature_extractor_seq_model = tf.keras.Sequential([
    InputLayer(input_shape = (CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], 3)),

    Conv2D(filters = 6, kernel_size = 3, strides=1, padding='valid', activation = 'relu', kernel_regularizer = L2(l2 = 0.01)),
    BatchNormalization(),
    MaxPool2D (pool_size = 2, strides= 2),

    Conv2D(filters = 16, kernel_size = 3, strides=1, padding='valid', activation = 'relu', kernel_regularizer = L2(l2 = 0.01)),
    BatchNormalization(),
    MaxPool2D (pool_size = 2, strides= 2),
])

In [17]:
feature_extractor_seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 6)       168       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 6)       24        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 6)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 16)      880       
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 16)      64        
 chNormalization)                                                
                                                      

### Callable Model (functional API)

In [18]:
func_input = Input(shape = (CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], 3), name = "Input Image")

x = feature_extractor_seq_model(func_input)

x = Flatten()(x)

x = Dense(100, activation = "relu", kernel_regularizer = L2(l2 = 0.01))(x)
x = Dropout(CONFIGURATION["DROPOUT_RATE"])(x)
x = BatchNormalization()(x)

x = Dense(10, activation = "relu", kernel_regularizer = L2(l2 = 0.01))(x)
x = Dropout(CONFIGURATION["DROPOUT_RATE"])(x)
x = BatchNormalization()(x)

func_output = Dense(1, activation = "sigmoid", kernel_regularizer = L2(l2 = 0.01))(x)

lenet_model = Model(func_input, func_output, name = "Lenet_Model")

In [19]:
lenet_model.summary()

Model: "Lenet_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 62, 62, 16)        1136      
                                                                 
 flatten (Flatten)           (None, 61504)             0         
                                                                 
 dense (Dense)               (None, 100)               6150500   
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 batch_normalization_2 (Bat  (None, 100)               400       
 chNormalization)                                                
                                                       

### Model training

In [20]:
lenet_model.compile(optimizer = Adam(learning_rate = CONFIGURATION['LEARNING_RATE']),
      loss = BinaryCrossentropy(CustomBCE(FACTOR)),        # Custom loss class defined above
      metrics = metrics)                                   # Metriics list defined above. For now the class that was defined for the

In [ ]:
history = lenet_model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = CONFIGURATION['N_EPOCHS'],
    verbose = 1,
    callbacks = [csv_callback, es_callback, scheduler_callback, plateau_callback]
    )


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


689/689 [==============================] - 52s 44ms/step - loss: 1.6803 - tp: 6777.0000 - fp: 3684.0000 - tn: 7339.0000 - fn: 4246.0000 - accuracy: 0.6403 - precision: 0.6478 - recall: 0.6148 - auc: 0.6891 - val_loss: 1.3653 - val_tp: 1174.0000 - val_fp: 471.0000 - val_tn: 923.0000 - val_fn: 187.0000 - val_accuracy: 0.7612 - val_precision: 0.7137 - val_recall: 0.8626 - val_auc: 0.8486 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/100
689/689 [==============================] - 37s 43ms/step - loss: 1.6303 - tp: 8463.0000 - fp: 2170.0000 - tn: 8852.0000 - fn: 2561.0000 - accuracy: 0.7854 - precision: 0.7959 - recall: 0.7677 - auc: 0.8565 - val_loss: 1.5839 - val_tp: 1247.0000 - val_fp: 417.0000 - val_tn: 949.0000 - val_fn: 142.0000 - val_accuracy: 0.7971 - val_precision: 0.7494 - val_recall: 0.8978 - val_auc: 0.9143 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0009048373904079199.
Epoch 3/100
688/689 [=====

https://www.pinecone.io/learn/regularization-in-neural-networks/

## Model Evaluation and Testing

In [ ]:
test_dataset = test_dataset.batch(1)

In [ ]:
lenet_model.evaluate(test_dataset)

In [ ]:
model.predict(test_dataset.take(1)[0][0])

In [ ]:
def parasite_or_not(x):
  if(x<0.5):
    return str("P")
  else:
    return str("U")

In [ ]:
parasite_or_not(model.predict(test_dataset.take(1)[0][0]))

In [ ]:
for i, (image, label) in enumerate(test_dataset.take(9)):

  ax = plt.subplot(3,3, i+1)
  plt.imshow(image[0])
  plt.title(str(parasite_or_not(label.numpy()[0])) + ":" + str(parasite_or_not(lente_model.predict(image)[0][0])))
  plt.axis("off")

### Hyperparameter tuning

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
feature_extractor_seq_model = tf.keras.Sequential([
    InputLayer(input_shape = (CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], 3)),

    Conv2D(filters = 6, kernel_size = 3, strides=1, padding='valid', activation = 'relu', kernel_regularizer = L2(l2 = 0.01)),
    BatchNormalization(),
    MaxPool2D (pool_size = 2, strides= 2),

    Conv2D(filters = 16, kernel_size = 3, strides=1, padding='valid', activation = 'relu', kernel_regularizer = L2(l2 = 0.01)),
    BatchNormalization(),
    MaxPool2D (pool_size = 2, strides= 2),
])

In [ ]:
func_input = Input(shape = (CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], 3), name = "Input Image")

x = feature_extractor_seq_model(func_input)

x = Flatten()(x)

x = Dense(100, activation = "relu", kernel_regularizer = L2(l2 = 0.01))(x)
x = Dropout(CONFIGURATION["DROPOUT_RATE"])(x)
x = BatchNormalization()(x)

x = Dense(10, activation = "relu", kernel_regularizer = L2(l2 = 0.01))(x)
x = Dropout(CONFIGURATION["DROPOUT_RATE"])(x)
x = BatchNormalization()(x)

func_output = Dense(1, activation = "sigmoid", kernel_regularizer = L2(l2 = 0.01))(x)

lenet_model = Model(func_input, func_output, name = "Lenet_Model")

In [ ]:
def model_builder(hp):
    # Define hyperparameters for the feature extractor layers
    num_conv_layers = hp.Int('num_conv_layers', min_value=1, max_value=3, default=2)
    num_filters = hp.Int('num_filters', min_value=8, max_value=64, step=8)
    filter_size = hp.Int('filter_size', min_value=3, max_value=5)
    conv_regularizer = hp.Choice('conv_regularizer', values=[0.0, 0.01, 0.001, 0.0001])

    num_dense_layers = hp.Int('num_dense_layers', min_value=1, max_value=3, default=2)
    dense_units = hp.Int('dense_units', min_value=32, max_value=512, step=32)
    dense_regularizer = hp.Choice('dense_regularizer', values=[0.0, 0.01, 0.001, 0.0001])

    output_regularizer = hp.Choice('output_regularizer', values=[0.0, 0.01, 0.001, 0.0001])

    dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)

    # Hyperparameters for Learning Rate Scheduler
    lr_schedule_factor = hp.Float('lr_schedule_factor', min_value=0.1, max_value=1.0, step=0.1)

    # Hyperparameters for Early Stopping
    early_stopping_patience = hp.Int('early_stopping_patience', min_value=5, max_value=20, step=5)

    # Hyperparameters for ReduceLROnPlateau
    reduce_lr_factor = hp.Float('reduce_lr_factor', min_value=0.1, max_value=1.0, step=0.1)
    reduce_lr_patience = hp.Int('reduce_lr_patience', min_value=2, max_value=10, step=1)
    reduce_lr_min_lr = hp.Float('reduce_lr_min_lr', min_value=1e-7, max_value=1e-3, step=1e-7)

    # Hyperparameters for LearningRate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])


    feature_extractor_seq_model = tf.keras.Sequential([
        InputLayer(input_shape=(CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], 3)),
    ])

    for _ in range(num_conv_layers):
      feature_extractor_seq_model.add(Conv2D(filters=num_filters, kernel_size=filter_size, strides=1, padding='valid', activation='relu', kernel_regularizer=L2(l2=conv_regularizer)))
      feature_extractor_seq_model.add(BatchNormalization())
      feature_extractor_seq_model.add(MaxPool2D(pool_size=2, strides=2))

    # Define hyperparameters for func_input layers
    func_input = Input(shape=(CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], 3), name="Input Image")
    x = feature_extractor_seq_model(func_input)
    x = Flatten()(x)

    for _ in range(num_dense_layers):
      x = Dense(dense_units, activation="relu", kernel_regularizer=L2(l2=0.01))(x)
      x = Dropout(CONFIGURATION["DROPOUT_RATE"])(x)
      x = BatchNormalization()(x)

    for _ in range(num_dense_layers):
      x = Dense(dense_units, activation="relu", kernel_regularizer=L2(l2=dense_regularizer))(x)
      x = Dropout(dropout_rate)(x)
      x = BatchNormalization()(x)

    func_output = Dense(1, activation="sigmoid", kernel_regularizer=L2(l2=output_regularizer))(x)

    model = Model(func_input, func_output, name="Lenet_Model")

    def lr_schedule(epoch):
      return learning_rate * lr_schedule_factor**epoch

    lr_scheduler = LearningRateScheduler(lr_schedule)

    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stopping_patience)

    reduce_lr_plateau = ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr_factor, patience=reduce_lr_patience, min_lr=reduce_lr_min_lr)

    callbacks = [lr_scheduler, early_stopping, reduce_lr_plateau]

    model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [ ]:
class MyModelTuner:
    def __init__(self, config):
        self.config = config

    def build_model(self, hp):
        # Define hyperparameters for the feature extractor layers
        num_conv_layers = hp.Int('num_conv_layers', min_value=1, max_value=3, default=2)
        num_filters = hp.Int('num_filters', min_value=8, max_value=64, step=8)
        filter_size = hp.Int('filter_size', min_value=3, max_value=5)
        conv_regularizer = hp.Choice('conv_regularizer', values=[0.0, 0.01, 0.001, 0.0001])

        num_dense_layers = hp.Int('num_dense_layers', min_value=1, max_value=3, default=2)
        dense_units = hp.Int('dense_units', min_value=32, max_value=512, step=32)
        dense_regularizer = hp.Choice('dense_regularizer', values=[0.0, 0.01, 0.001, 0.0001])

        dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)

        output_regularizer = hp.Choice('output_regularizer', values=[0.0, 0.01, 0.001, 0.0001])

        # Define hyperparameters for Learning Rate Scheduler
        lr_schedule_factor = hp.Float('lr_schedule_factor', min_value=0.1, max_value=1.0, step=0.1)

        # Define hyperparameters for Early Stopping
        early_stopping_patience = hp.Int('early_stopping_patience', min_value=5, max_value=20, step=5)

        # Define hyperparameters for ReduceLROnPlateau
        reduce_lr_factor = hp.Float('reduce_lr_factor', min_value=0.1, max_value=1.0, step=0.1)
        reduce_lr_patience = hp.Int('reduce_lr_patience', min_value=2, max_value=10, step=1)
        reduce_lr_min_lr = hp.Float('reduce_lr_min_lr', min_value=1e-7, max_value=1e-3, step=1e-7)

        feature_extractor_seq_model = tf.keras.Sequential([
            InputLayer(input_shape=(self.config["IM_HEIGHT"], self.config["IM_WIDTH"], 3)),
        ])

        for _ in range(num_conv_layers):
            feature_extractor_seq_model.add(Conv2D(filters=num_filters, kernel_size=filter_size, strides=1, padding='valid', activation='relu', kernel_regularizer=L2(l2=conv_regularizer)))
            feature_extractor_seq_model.add(BatchNormalization())
            feature_extractor_seq_model.add(MaxPool2D(pool_size=2, strides=2))

        func_input = Input(shape=(self.config["IM_HEIGHT"], self.config["IM_WIDTH"], 3), name="Input Image")
        x = feature_extractor_seq_model(func_input)
        x = Flatten()(x)

        for _ in range(num_dense_layers):
            x = Dense(dense_units, activation="relu", kernel_regularizer=L2(l2=dense_regularizer))(x)
            x = Dropout(dropout_rate)(x)
            x = BatchNormalization()(x)

        func_output = Dense(1, activation="sigmoid", kernel_regularizer=L2(l2=output_regularizer))(x)

        # Add learning rate as a hyperparameter
        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

        model = Model(func_input, func_output, name="Lenet_Model")

        # Configure Learning Rate Scheduler based on hyperparameters
        def lr_schedule(epoch):
            return learning_rate * lr_schedule_factor**epoch

        lr_scheduler = LearningRateScheduler(lr_schedule)

        # Configure Early Stopping based on hyperparameters
        early_stopping = EarlyStopping(monitor='val_loss', patience=early_stopping_patience)

        # Configure ReduceLROnPlateau based on hyperparameters
        reduce_lr_plateau = ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr_factor, patience=reduce_lr_patience, min_lr=reduce_lr_min_lr)

        callbacks = [lr_scheduler, early_stopping, reduce_lr_plateau]

        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        return model

    def run_tuner(self):
        # Create a Keras Tuner RandomSearch tuner
        tuner = RandomSearch(
            self.build_model,
            objective='val_accuracy',
            max_trials=self.config["max_trials"],
            directory='my_dir',
            project_name='my_project'
        )

        # Define the search space and perform hyperparameter tuning
        tuner.search(x=self.config["x_train"],
                     y=self.config["y_train"],
                     validation_data=(self.config["x_val"], self.config["y_val"]),
                     epochs=self.config["epochs"])

        # Get the best hyperparameters
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

        return best_hps


# Example configuration
config = {
    "IM_HEIGHT": 128,
    "IM_WIDTH": 128,
    "max_trials": 10,
    "x_train": np.random.rand(100, 128, 128, 3),
    "y_train": np.random.randint(0, 2, size=(100,)),
    "x_val": np.random.rand(20, 128, 128, 3),
    }

# Create an instance of MyModelTuner
my_tuner = MyModelTuner(config)

# Run the hyperparameter tuning process
best_hyperparameters = my_tuner.run_tuner()

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_hyperparameters)